# Metrics calculation (LLM)

In [15]:
import pandas as pd
# summary_dataset = pd.read_csv("summary_dataset_with_bertscore.csv")
# columns = [column for column in summary_dataset.columns if "bertscore" not in column] 
# print(columns)
# summary_dataset = summary_dataset[columns]
# summary_dataset.to_csv("summary_dataset.csv", index=False)
# summary_dataset = pd.read_csv("summary_dataset.csv")
# summary_dataset.head()

In [16]:
!pwd

/home/skatori/Work/MasDiploma


In [17]:
summary_dataset = pd.read_csv("summary_dataset_with_bertscore_bleu_rouge_meteor.csv")
summary_dataset = summary_dataset.rename(columns={"Unnamed: 0": "Ind"})
import random
# print(len(summary_dataset))
random.seed(42)
indices_for_evaluation = random.sample(range(0, len(summary_dataset)), 100)
indices_for_evaluation = sorted(indices_for_evaluation)
summary_dataset = summary_dataset[summary_dataset["Ind"].isin(indices_for_evaluation)]

In [18]:
summary_files = [
    "llama_7b_predictions.csv",
    "starling_predictions.csv",
    "yagpt_predictions.csv",
    "yagpt3_predictions.csv"
]

llm_data = pd.DataFrame()
for summary_file in summary_files:
    model_name = summary_file.split(".")[0]
    df = pd.read_csv(summary_file)[[model_name]]
    llm_data = pd.concat([llm_data, df], axis=1)

llm_data.head()

,llama_7b_predictions,starling_predictions,yagpt_predictions,yagpt3_predictions
0,"Bluetooth постоянно включен, это опасно, так к...",Постоянно включенный Bluetooth может создавать...,- Постоянно включенный Bluetooth на смартфоне ...,Доцент кафедры информатики РЭУ им. Плеханова А...
1,Китайское издание Sina высоко оценило новый ро...,Китайский издание Sina высоко оценило новый ро...,"К сожалению, я не могу ничего сказать об этом....","К сожалению, я не могу ничего сказать об этом...."
2,8 сентября в Японии началась избирательная кам...,"Всего три кандидата — Есихидэ Суга, Сигэру Иси...","К сожалению, я не могу ничего сказать об этом....","К сожалению, я не могу ничего сказать об этом...."
3,"Мария Колесникова, член президиума координацио...","Мария Колесникова, член президиума координацио...",- Член президиума координационного совета оппо...,"Мария Колесникова, член президиума координацио..."
4,Аналитики Deutsche Bank предсказали наступлени...,"Аналитики Deutsche Bank предполагают, что 2020...",- Аналитики Deutsche Bank предсказали наступле...,Аналитики Deutsche Bank предсказали наступлени...


In [19]:
summary_dataset = pd.concat([summary_dataset, llm_data.set_index(summary_dataset.index)], axis=1)
print(summary_dataset.columns)
summary_dataset = summary_dataset[[column for column in summary_dataset.columns if "bertscore" not in column and "rouge" not in column and "meteor" not in column]]
summary_dataset = summary_dataset[summary_dataset["yagpt_predictions"] != "К сожалению, я не могу ничего сказать об этом. Давайте сменим тему?"]
summary_dataset.head()
print("Total: ", len(summary_dataset))

Index(['Unnamed: 0.1', 'Ind', 'title', 'text', 'summary', 'mbart_predictions',
       'mt5_predictions', 'summarunner_predictions',
       'mbart_predictions_bertscore_precision',
       'mbart_predictions_bertscore_recall', 'mbart_predictions_bertscore_f1',
       'mt5_predictions_bertscore_precision',
       'mt5_predictions_bertscore_recall', 'mt5_predictions_bertscore_f1',
       'summarunner_predictions_bertscore_precision',
       'summarunner_predictions_bertscore_recall',
       'summarunner_predictions_bertscore_f1',
       'mbart_predictions_bertscore_precision.1',
       'mbart_predictions_bertscore_recall.1',
       'mbart_predictions_bertscore_f1.1',
       'mt5_predictions_bertscore_precision.1',
       'mt5_predictions_bertscore_recall.1', 'mt5_predictions_bertscore_f1.1',
       'summarunner_predictions_bertscore_precision.1',
       'summarunner_predictions_bertscore_recall.1',
       'summarunner_predictions_bertscore_f1.1', 'mbart_predictions_rouge1',
       'mbart_p

## BERTScore

In [20]:
from evaluate import load
bertscore = load("bertscore")
files = [
    "mbart_predictions.txt",
    "mt5_predictions.txt",
    "summarunner_predictions.txt",
    "llama_7b_predictions.csv",
    "starling_predictions.csv",
    "yagpt_predictions.csv",
    "yagpt3_predictions.csv"
]

In [21]:
results = {}
references = list(summary_dataset["summary"])
for filename in files:
    predictions_field = filename.split(".")[0]
    predictions = list(summary_dataset[predictions_field])
    results[predictions_field] = bertscore.compute(predictions=predictions, references=references, lang="ru", verbose=True, model_type="microsoft/mdeberta-v3-base", nthreads=16, batch_size=16, use_fast_tokenizer=False)

calculating scores...
computing bert embedding.


  0%|          | 0/9 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/5 [00:00<?, ?it/s]

done in 369593.78 seconds, 0.00 sentences/sec
calculating scores...
computing bert embedding.


...

done in 369595.54 seconds, 0.00 sentences/sec
calculating scores...
computing bert embedding.


  0%|          | 0/9 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/5 [00:00<?, ?it/s]

done in 369595.92 seconds, 0.00 sentences/sec


In [22]:
data_keys = list(results.keys())
for data_key in data_keys:
    old_keys = list(results[data_key].keys())
    for old_metric_key in old_keys:
        results[data_key][f"{data_key}_bertscore_" + old_metric_key] = results[data_key].pop(old_metric_key)

In [23]:
print(results[filename.split(".")[0]].keys())
print(len(results[filename.split(".")[0]]))

dict_keys(['yagpt3_predictions_bertscore_precision', 'yagpt3_predictions_bertscore_recall', 'yagpt3_predictions_bertscore_f1', 'yagpt3_predictions_bertscore_hashcode'])
4


In [24]:
print(len(results[filename.split(".")[0]][f"{filename.split('.')[0]}_bertscore_precision"]))

65


In [25]:
new_data = {}
for data_key in results.keys():
    for metric_key in results[data_key].keys():
        if "hashcode" in metric_key:
            continue
        new_data[metric_key] = pd.Series(results[data_key][metric_key])
new_data_df = pd.DataFrame(new_data)
new_data_df.head()

,mbart_predictions_bertscore_precision,mbart_predictions_bertscore_recall,mbart_predictions_bertscore_f1,mt5_predictions_bertscore_precision,mt5_predictions_bertscore_recall,mt5_predictions_bertscore_f1,summarunner_predictions_bertscore_precision,summarunner_predictions_bertscore_recall,summarunner_predictions_bertscore_f1,llama_7b_predictions_bertscore_precision,...,llama_7b_predictions_bertscore_f1,starling_predictions_bertscore_precision,starling_predictions_bertscore_recall,starling_predictions_bertscore_f1,yagpt_predictions_bertscore_precision,yagpt_predictions_bertscore_recall,yagpt_predictions_bertscore_f1,yagpt3_predictions_bertscore_precision,yagpt3_predictions_bertscore_recall,yagpt3_predictions_bertscore_f1
0,0.634706,0.662176,0.648150,0.697221,0.585219,0.636329,0.618635,0.642960,0.630563,0.626199,...,0.632462,0.671103,0.657498,0.664231,0.655310,0.665903,0.660564,0.605509,0.668242,0.635331
1,0.779796,0.720445,0.748946,0.819961,0.630633,0.712942,0.667503,0.665685,0.666593,0.662520,...,0.667891,0.693992,0.739694,0.716114,0.673741,0.731413,0.701393,0.636105,0.701595,0.667247
2,0.775955,0.711766,0.742476,0.750154,0.590834,0.661030,0.669023,0.639625,0.653994,0.649071,...,0.689116,0.695308,0.682452,0.688820,0.657999,0.705884,0.681101,0.677816,0.736740,0.706050
3,0.695167,0.724142,0.709359,0.659730,0.614271,0.636190,0.674246,0.654754,0.664357,0.634675,...,0.697208,0.665577,0.761002,0.710098,0.653899,0.781621,0.712078,0.668633,0.727307,0.696737
4,0.681680,0.608897,0.643236,0.651160,0.601042,0.625098,0.618116,0.594370,0.606011,0.637715,...,0.657648,0.632436,0.662249,0.646999,0.659560,0.697301,0.677906,0.650176,0.691115,0.670020


In [27]:
new_summary_dataset = pd.concat([summary_dataset, new_data_df.set_index(summary_dataset.index)], axis=1)
new_summary_dataset.head()
print(len(new_summary_dataset))

65


In [28]:
summary_dataset = pd.DataFrame(new_summary_dataset)
del new_summary_dataset

In [29]:
summary_dataset.head()
print(len(summary_dataset))

65


In [30]:
from statistics import mean 
from collections import defaultdict
score_stats = defaultdict(dict) if (not "score_stats" in locals() and not "score_stats" in globals()) else score_stats
for filename in files:
    predictions_field = filename.split(".")[0]
    
    mean_f1 = mean(results[predictions_field][f"{predictions_field}_bertscore_f1"])
    var_f1 = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_bertscore_f1"])) - mean_f1 ** 2
    
    mean_precision = mean(results[predictions_field][f"{predictions_field}_bertscore_precision"])
    var_precision = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_bertscore_precision"])) - mean_precision ** 2
    
    mean_recall = mean(results[predictions_field][f"{predictions_field}_bertscore_recall"])
    var_recall = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_bertscore_recall"])) - mean_recall ** 2
    
    score_stats[predictions_field].update({"mean_precision": mean_precision, "var_precision": var_precision, "mean_recall": mean_recall, "var_recall": var_recall, "mean_f1": mean_f1, "var_f1": var_f1})
print(dict(score_stats))

{'mbart_predictions': {'mean_precision': 0.6900501682208134, 'var_precision': 0.0019415853485905687, 'mean_recall': 0.6960230689782363, 'var_recall': 0.0025041195684459283, 'mean_f1': 0.6922733288544874, 'var_f1': 0.00170587457440724}, 'mt5_predictions': {'mean_precision': 0.7098548559042124, 'var_precision': 0.003583850681936007, 'mean_recall': 0.6099580485087175, 'var_recall': 0.0028486138733002653, 'mean_f1': 0.6551174383897048, 'var_f1': 0.002486519453542002}, 'summarunner_predictions': {'mean_precision': 0.6632669393832867, 'var_precision': 0.002033478402692912, 'mean_recall': 0.6764815312165481, 'var_recall': 0.0026100377107953765, 'mean_f1': 0.6691702916071965, 'var_f1': 0.0018536017157450568}, 'llama_7b_predictions': {'mean_precision': 0.6443997227228605, 'var_precision': 0.0008212487075641706, 'mean_recall': 0.7223757973084083, 'var_recall': 0.001855303761597038, 'mean_f1': 0.680531212916741, 'var_f1': 0.0008262458785404658}, 'starling_predictions': {'mean_precision': 0.665734

In [31]:
results.keys()

dict_keys(['mbart_predictions', 'mt5_predictions', 'summarunner_predictions', 'llama_7b_predictions', 'starling_predictions', 'yagpt_predictions', 'yagpt3_predictions'])

In [32]:
stats_pd = pd.DataFrame(score_stats)

In [33]:
stats_pd.head(None)

,mbart_predictions,mt5_predictions,summarunner_predictions,llama_7b_predictions,starling_predictions,yagpt_predictions,yagpt3_predictions
mean_precision,0.690050,0.709855,0.663267,0.644400,0.665735,0.657500,0.659426
var_precision,0.001942,0.003584,0.002033,0.000821,0.002084,0.000793,0.001332
mean_recall,0.696023,0.609958,0.676482,0.722376,0.706216,0.714787,0.731001
var_recall,0.002504,0.002849,0.002610,0.001855,0.001771,0.001589,0.002079
mean_f1,0.692273,0.655117,0.669170,0.680531,0.684643,0.684335,0.692904
var_f1,0.001706,0.002487,0.001854,0.000826,0.001501,0.000723,0.001329


In [34]:
summary_dataset.to_csv("summary_dataset_with_bertscore.csv", index=False)

## BLEU

In [35]:
import razdel
from evaluate import load
from tqdm import tqdm
import nltk

# results = {}
bleu = load('bleu')
references = list(summary_dataset["summary"])

def a(x):
    tokens = list(razdel.tokenize(x))
    return [_.text for _ in tokens]
    # return x.split()

for filename in files:
    predictions_field = filename.split(".")[0]
    predictions = list(summary_dataset[predictions_field])
    # results[predictions_field] = rouge.compute(predictions=predictions, references=references, use_aggregator=False, tokenizer=lambda x : list(razdel.tokenize(x)))
    results[predictions_field]["bleu"] = []
    for ref, pred in tqdm(zip(references, predictions)):
        bl = bleu.compute(predictions=[pred], references=[ref], tokenizer=a, smooth=True)
        # bl = nltk.translate.bleu(pred, ref)
        results[predictions_field]["bleu"].append(bl["bleu"])

65it [00:00, 584.29it/s]
65it [00:00, 246.10it/s]
65it [00:00, 562.92it/s]
65it [00:00, 494.92it/s]
65it [00:00, 531.21it/s]
65it [00:00, 515.82it/s]
65it [00:00, 518.49it/s]


In [36]:
data_keys = list(results.keys())
print(data_keys)
for data_key in data_keys:
    old_keys = list(results[data_key].keys())
    print(old_keys)
    for old_metric_key in old_keys:
        results[data_key][f"{data_key}_" + old_metric_key] = results[data_key].pop(old_metric_key)

['mbart_predictions', 'mt5_predictions', 'summarunner_predictions', 'llama_7b_predictions', 'starling_predictions', 'yagpt_predictions', 'yagpt3_predictions']
['mbart_predictions_bertscore_precision', 'mbart_predictions_bertscore_recall', 'mbart_predictions_bertscore_f1', 'mbart_predictions_bertscore_hashcode', 'bleu']
['mt5_predictions_bertscore_precision', 'mt5_predictions_bertscore_recall', 'mt5_predictions_bertscore_f1', 'mt5_predictions_bertscore_hashcode', 'bleu']
['summarunner_predictions_bertscore_precision', 'summarunner_predictions_bertscore_recall', 'summarunner_predictions_bertscore_f1', 'summarunner_predictions_bertscore_hashcode', 'bleu']
['llama_7b_predictions_bertscore_precision', 'llama_7b_predictions_bertscore_recall', 'llama_7b_predictions_bertscore_f1', 'llama_7b_predictions_bertscore_hashcode', 'bleu']
['starling_predictions_bertscore_precision', 'starling_predictions_bertscore_recall', 'starling_predictions_bertscore_f1', 'starling_predictions_bertscore_hashcode',

In [37]:
print(results[filename.split(".")[0]].keys())

dict_keys(['yagpt3_predictions_yagpt3_predictions_bertscore_precision', 'yagpt3_predictions_yagpt3_predictions_bertscore_recall', 'yagpt3_predictions_yagpt3_predictions_bertscore_f1', 'yagpt3_predictions_yagpt3_predictions_bertscore_hashcode', 'yagpt3_predictions_bleu'])


In [38]:
print(results[filename.split(".")[0]][f"{filename.split('.')[0]}_bleu"])

[0.019104499120758397, 0.02447167872800763, 0.05608640868147403, 0.061944612569869696, 0.04279592866572136, 0.12903758633524678, 0.027968360509611868, 0.06793375012047573, 0.17060928518931617, 0.02120207527968304, 0.06865522713382077, 0.08097559540055488, 0.0699363486683451, 0.06882160869839149, 0.12885119253120664, 0.021818903131420007, 0.022829847423290653, 0.04269234003902816, 0.045157140680870546, 0.03873582507075529, 0.01777724594642383, 0.02874368217704366, 0.07391908926173968, 0.03435763505835022, 0.07666911020597449, 0.056093243250242925, 0.05964418182268273, 0.04425184981184798, 0.023570851710527026, 0.03144406647233181, 0.027620227857333722, 0.022005485861068718, 0.027903786634678848, 0.0811563673673393, 0.04811212639299925, 0.027471318708441127, 0.11090689733307425, 0.06725110619469749, 0.03006347925387719, 0.0959471190220409, 0.10449095130554766, 0.03574613978248222, 0.02734195277682282, 0.03240113904384493, 0.031069874854278538, 0.028440172000217884, 0.038524408605448764, 

In [39]:
new_data = {}
for data_key in results.keys():
    for metric_key in results[data_key].keys():
        if not "bleu" in metric_key:
            continue
        new_data[metric_key] = pd.Series(results[data_key][metric_key])
new_data_df = pd.DataFrame(new_data)
new_data_df.head()

,mbart_predictions_bleu,mt5_predictions_bleu,summarunner_predictions_bleu,llama_7b_predictions_bleu,starling_predictions_bleu,yagpt_predictions_bleu,yagpt3_predictions_bleu
0,0.036231,0.012091,0.061176,0.036385,0.042085,0.023872,0.019104
1,0.082445,0.007061,0.043380,0.032554,0.052262,0.083561,0.024472
2,0.133905,0.015023,0.088968,0.025706,0.062624,0.037451,0.056086
3,0.059402,0.034030,0.072320,0.023837,0.043465,0.028014,0.061945
4,0.031277,0.023400,0.028119,0.041259,0.040176,0.038714,0.042796


In [40]:
new_summary_dataset = pd.concat([summary_dataset, new_data_df.set_index(summary_dataset.index)], axis=1)
new_summary_dataset.head()

,Unnamed: 0.1,Ind,title,text,summary,mbart_predictions,mt5_predictions,summarunner_predictions,llama_7b_predictions,starling_predictions,...,yagpt3_predictions_bertscore_precision,yagpt3_predictions_bertscore_recall,yagpt3_predictions_bertscore_f1,mbart_predictions_bleu,mt5_predictions_bleu,summarunner_predictions_bleu,llama_7b_predictions_bleu,starling_predictions_bleu,yagpt_predictions_bleu,yagpt3_predictions_bleu
53,53,53,Названа опасность постоянно включенного Blueto...,Постоянно включенный Bluetooth на смартфоне гр...,Активированный в смартфоне Bluetooth может пре...,Постоянно включенный Bluetooth на смартфоне гр...,"Эксперты предупреждают о том, что отключать Bl...",постоянно включенный bluetooth на смартфоне гр...,"Bluetooth постоянно включен, это опасно, так к...",Постоянно включенный Bluetooth может создавать...,...,0.605509,0.668242,0.635331,0.036231,0.012091,0.061176,0.036385,0.042085,0.023872,0.019104
244,244,244,Колесникова проходит подозреваемой по делу о з...,Член президиума координационного совета оппози...,Члена президиума оппозиционного Координационно...,Член президиума координационного совета оппози...,Член президиума координационного совета оппози...,член президиума координационного совета оппози...,"Мария Колесникова, член президиума координацио...","Мария Колесникова, член президиума координацио...",...,0.636105,0.701595,0.667247,0.082445,0.007061,0.043380,0.032554,0.052262,0.083561,0.024472
260,260,260,Deutsche Bank: в мире наступает эпоха беспорядка,Аналитики Deutsche Bank обнародовали исследова...,В истории человечества наступает эпоха беспоря...,В 2020 году в мире наступит новая эпоха беспор...,Пандемия коронавируса привела к появлениям нов...,аналитики deutsche bank обнародовали исследова...,Аналитики Deutsche Bank предсказали наступлени...,"Аналитики Deutsche Bank предполагают, что 2020...",...,0.677816,0.736740,0.706050,0.133905,0.015023,0.088968,0.025706,0.062624,0.037451,0.056086
355,355,355,«Ъ»: Минтранс подготовил проект поправок к ПДД,ГИБДД совместно с Минтрансом разработала масшт...,Масштабный проект изменений в ПДД подготовили ...,ГИБДД совместно с Минтрансом разработала масшт...,В Госдуму вступили в силу поправки в правила д...,гибдд совместно с минтрансом разработала масшт...,ГИБДД и Минтранс разработали проект изменений ...,ГИБДД и Минтранс разработали проект изменений ...,...,0.668633,0.727307,0.696737,0.059402,0.034030,0.072320,0.023837,0.043465,0.028014,0.061945
653,653,653,Tesla выпустит бюджетный беспилотник в 2023 году,Вечером 22 сентября на конференции Battery Day...,Миллиардер Илон Маск пообещал представить бюдж...,На конференции Battery Day глава Tesla Илон Ма...,Илон Маск рассчитывает вывести на рынок бюджет...,вечером 22 сентября на конференции battery day...,Илон Маск представил планы Tesla по выходу на ...,"В ходе конференции Tesla Battery Day, Илон Мас...",...,0.650176,0.691115,0.670020,0.031277,0.023400,0.028119,0.041259,0.040176,0.038714,0.042796


In [41]:
summary_dataset = pd.DataFrame(new_summary_dataset)
del new_summary_dataset

In [42]:
summary_dataset.head()

,Unnamed: 0.1,Ind,title,text,summary,mbart_predictions,mt5_predictions,summarunner_predictions,llama_7b_predictions,starling_predictions,...,yagpt3_predictions_bertscore_precision,yagpt3_predictions_bertscore_recall,yagpt3_predictions_bertscore_f1,mbart_predictions_bleu,mt5_predictions_bleu,summarunner_predictions_bleu,llama_7b_predictions_bleu,starling_predictions_bleu,yagpt_predictions_bleu,yagpt3_predictions_bleu
53,53,53,Названа опасность постоянно включенного Blueto...,Постоянно включенный Bluetooth на смартфоне гр...,Активированный в смартфоне Bluetooth может пре...,Постоянно включенный Bluetooth на смартфоне гр...,"Эксперты предупреждают о том, что отключать Bl...",постоянно включенный bluetooth на смартфоне гр...,"Bluetooth постоянно включен, это опасно, так к...",Постоянно включенный Bluetooth может создавать...,...,0.605509,0.668242,0.635331,0.036231,0.012091,0.061176,0.036385,0.042085,0.023872,0.019104
244,244,244,Колесникова проходит подозреваемой по делу о з...,Член президиума координационного совета оппози...,Члена президиума оппозиционного Координационно...,Член президиума координационного совета оппози...,Член президиума координационного совета оппози...,член президиума координационного совета оппози...,"Мария Колесникова, член президиума координацио...","Мария Колесникова, член президиума координацио...",...,0.636105,0.701595,0.667247,0.082445,0.007061,0.043380,0.032554,0.052262,0.083561,0.024472
260,260,260,Deutsche Bank: в мире наступает эпоха беспорядка,Аналитики Deutsche Bank обнародовали исследова...,В истории человечества наступает эпоха беспоря...,В 2020 году в мире наступит новая эпоха беспор...,Пандемия коронавируса привела к появлениям нов...,аналитики deutsche bank обнародовали исследова...,Аналитики Deutsche Bank предсказали наступлени...,"Аналитики Deutsche Bank предполагают, что 2020...",...,0.677816,0.736740,0.706050,0.133905,0.015023,0.088968,0.025706,0.062624,0.037451,0.056086
355,355,355,«Ъ»: Минтранс подготовил проект поправок к ПДД,ГИБДД совместно с Минтрансом разработала масшт...,Масштабный проект изменений в ПДД подготовили ...,ГИБДД совместно с Минтрансом разработала масшт...,В Госдуму вступили в силу поправки в правила д...,гибдд совместно с минтрансом разработала масшт...,ГИБДД и Минтранс разработали проект изменений ...,ГИБДД и Минтранс разработали проект изменений ...,...,0.668633,0.727307,0.696737,0.059402,0.034030,0.072320,0.023837,0.043465,0.028014,0.061945
653,653,653,Tesla выпустит бюджетный беспилотник в 2023 году,Вечером 22 сентября на конференции Battery Day...,Миллиардер Илон Маск пообещал представить бюдж...,На конференции Battery Day глава Tesla Илон Ма...,Илон Маск рассчитывает вывести на рынок бюджет...,вечером 22 сентября на конференции battery day...,Илон Маск представил планы Tesla по выходу на ...,"В ходе конференции Tesla Battery Day, Илон Мас...",...,0.650176,0.691115,0.670020,0.031277,0.023400,0.028119,0.041259,0.040176,0.038714,0.042796


In [43]:
# from statistics import mean 
# from collections import defaultdict
# score_stats = defaultdict(dict) if (not "score_stats" in locals() and not "score_stats" in globals()) else score_stats
# for filename in files:
#     predictions_field = filename.split(".")[0]
#     mean_rouge1 = mean(results[predictions_field][f"{predictions_field}_rouge1"])
#     var_rouge1 = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_rouge1"])) - mean_rouge1 ** 2
    
#     mean_rouge2 = mean(results[predictions_field][f"{predictions_field}_rouge2"])
#     var_rouge2 = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_rouge2"])) - mean_rouge2 ** 2
    
#     mean_rougeL = mean(results[predictions_field][f"{predictions_field}_rougeL"])
#     var_rougeL = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_rougeL"])) - mean_rougeL ** 2
    
#     mean_rougeLsum = mean(results[predictions_field][f"{predictions_field}_rougeLsum"])
#     var_rougeLsum = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_rougeLsum"])) - mean_rougeLsum ** 2
#     score_stats[predictions_field].update({"mean_rouge1": mean_rouge1, "var_rouge1": var_rouge1, "mean_rouge2": mean_rouge2, "var_rouge2": var_rouge2, "mean_rougeL": mean_rougeL, "var_rougeL": var_rougeL, "mean_rougeLsum": mean_rougeLsum, "var_rougeLsum": var_rougeLsum})
# print(dict(score_stats))

In [44]:
# stats_pd = pd.DataFrame(score_stats)

In [45]:
# stats_pd.head(len(stats_pd))

In [46]:
summary_dataset.to_csv("summary_dataset_with_bertscore_bleu.csv", index=False)

In [48]:
print(len(summary_dataset))

65


## ROUGE

In [49]:
import pandas as pd
summary_dataset = pd.read_csv("summary_dataset_with_bertscore_bleu.csv")
files = [
    "mbart_predictions.txt",
    "mt5_predictions.txt",
    "summarunner_predictions.txt",
    "llama_7b_predictions.csv",
    "starling_predictions.csv",
    "yagpt_predictions.csv",
    "yagpt3_predictions.csv"
]

In [50]:
import razdel
from evaluate import load
# results = {}
rouge = load('rouge')
references = list(summary_dataset["summary"])

def a(x):
    tokens = list(razdel.tokenize(x))
    return [_.text for _ in tokens]
    # return x.split()

for filename in files:
    predictions_field = filename.split(".")[0]
    predictions = list(summary_dataset[predictions_field])
    # results[predictions_field] = rouge.compute(predictions=predictions, references=references, use_aggregator=False, tokenizer=lambda x : list(razdel.tokenize(x)))
    results[predictions_field] = rouge.compute(predictions=predictions, references=references, use_aggregator=False, tokenizer=a)

In [51]:
data_keys = list(results.keys())
for data_key in data_keys:
    old_keys = list(results[data_key].keys())
    for old_metric_key in old_keys:
        results[data_key][f"{data_key}_" + old_metric_key] = results[data_key].pop(old_metric_key)

In [52]:
print(results[filename.split(".")[0]].keys())

dict_keys(['yagpt3_predictions_rouge1', 'yagpt3_predictions_rouge2', 'yagpt3_predictions_rougeL', 'yagpt3_predictions_rougeLsum'])


In [53]:
print(len(results[filename.split(".")[0]][f"{filename.split('.')[0]}_rouge1"]))

65


In [54]:
new_data = {}
for data_key in results.keys():
    for metric_key in results[data_key].keys():
        if "hashcode" in metric_key:
            continue
        new_data[metric_key] = pd.Series(results[data_key][metric_key])
new_data_df = pd.DataFrame(new_data)
new_data_df.head()

,mbart_predictions_rouge1,mbart_predictions_rouge2,mbart_predictions_rougeL,mbart_predictions_rougeLsum,mt5_predictions_rouge1,mt5_predictions_rouge2,mt5_predictions_rougeL,mt5_predictions_rougeLsum,summarunner_predictions_rouge1,summarunner_predictions_rouge2,...,starling_predictions_rougeL,starling_predictions_rougeLsum,yagpt_predictions_rouge1,yagpt_predictions_rouge2,yagpt_predictions_rougeL,yagpt_predictions_rougeLsum,yagpt3_predictions_rouge1,yagpt3_predictions_rouge2,yagpt3_predictions_rougeL,yagpt3_predictions_rougeLsum
0,0.228070,0.017857,0.157895,0.157895,0.151515,0.000000,0.090909,0.090909,0.262626,0.041237,...,0.162162,0.162162,0.176101,0.025478,0.125786,0.138365,0.171429,0.011561,0.102857,0.114286
1,0.368932,0.118812,0.330097,0.330097,0.202532,0.025974,0.202532,0.202532,0.269841,0.032258,...,0.256410,0.256410,0.310160,0.086486,0.235294,0.171123,0.257732,0.031250,0.164948,0.175258
2,0.395604,0.179775,0.351648,0.351648,0.176471,0.090909,0.147059,0.147059,0.372549,0.060000,...,0.214286,0.214286,0.241546,0.058537,0.164251,0.164251,0.309677,0.078431,0.193548,0.245161
3,0.333333,0.063830,0.208333,0.208333,0.169492,0.000000,0.169492,0.169492,0.233766,0.053333,...,0.174497,0.174497,0.239130,0.054945,0.173913,0.173913,0.317757,0.095238,0.224299,0.205607
4,0.175824,0.022472,0.131868,0.131868,0.195122,0.025000,0.170732,0.170732,0.142857,0.000000,...,0.198473,0.198473,0.321678,0.056738,0.223776,0.223776,0.285714,0.108434,0.190476,0.238095


In [55]:
new_summary_dataset = pd.concat([summary_dataset, new_data_df.set_index(summary_dataset.index)], axis=1)
new_summary_dataset.head()

,Unnamed: 0.1,Ind,title,text,summary,mbart_predictions,mt5_predictions,summarunner_predictions,llama_7b_predictions,starling_predictions,...,starling_predictions_rougeL,starling_predictions_rougeLsum,yagpt_predictions_rouge1,yagpt_predictions_rouge2,yagpt_predictions_rougeL,yagpt_predictions_rougeLsum,yagpt3_predictions_rouge1,yagpt3_predictions_rouge2,yagpt3_predictions_rougeL,yagpt3_predictions_rougeLsum
0,53,53,Названа опасность постоянно включенного Blueto...,Постоянно включенный Bluetooth на смартфоне гр...,Активированный в смартфоне Bluetooth может пре...,Постоянно включенный Bluetooth на смартфоне гр...,"Эксперты предупреждают о том, что отключать Bl...",постоянно включенный bluetooth на смартфоне гр...,"Bluetooth постоянно включен, это опасно, так к...",Постоянно включенный Bluetooth может создавать...,...,0.162162,0.162162,0.176101,0.025478,0.125786,0.138365,0.171429,0.011561,0.102857,0.114286
1,244,244,Колесникова проходит подозреваемой по делу о з...,Член президиума координационного совета оппози...,Члена президиума оппозиционного Координационно...,Член президиума координационного совета оппози...,Член президиума координационного совета оппози...,член президиума координационного совета оппози...,"Мария Колесникова, член президиума координацио...","Мария Колесникова, член президиума координацио...",...,0.256410,0.256410,0.310160,0.086486,0.235294,0.171123,0.257732,0.031250,0.164948,0.175258
2,260,260,Deutsche Bank: в мире наступает эпоха беспорядка,Аналитики Deutsche Bank обнародовали исследова...,В истории человечества наступает эпоха беспоря...,В 2020 году в мире наступит новая эпоха беспор...,Пандемия коронавируса привела к появлениям нов...,аналитики deutsche bank обнародовали исследова...,Аналитики Deutsche Bank предсказали наступлени...,"Аналитики Deutsche Bank предполагают, что 2020...",...,0.214286,0.214286,0.241546,0.058537,0.164251,0.164251,0.309677,0.078431,0.193548,0.245161
3,355,355,«Ъ»: Минтранс подготовил проект поправок к ПДД,ГИБДД совместно с Минтрансом разработала масшт...,Масштабный проект изменений в ПДД подготовили ...,ГИБДД совместно с Минтрансом разработала масшт...,В Госдуму вступили в силу поправки в правила д...,гибдд совместно с минтрансом разработала масшт...,ГИБДД и Минтранс разработали проект изменений ...,ГИБДД и Минтранс разработали проект изменений ...,...,0.174497,0.174497,0.239130,0.054945,0.173913,0.173913,0.317757,0.095238,0.224299,0.205607
4,653,653,Tesla выпустит бюджетный беспилотник в 2023 году,Вечером 22 сентября на конференции Battery Day...,Миллиардер Илон Маск пообещал представить бюдж...,На конференции Battery Day глава Tesla Илон Ма...,Илон Маск рассчитывает вывести на рынок бюджет...,вечером 22 сентября на конференции battery day...,Илон Маск представил планы Tesla по выходу на ...,"В ходе конференции Tesla Battery Day, Илон Мас...",...,0.198473,0.198473,0.321678,0.056738,0.223776,0.223776,0.285714,0.108434,0.190476,0.238095


In [56]:
summary_dataset = pd.DataFrame(new_summary_dataset)
del new_summary_dataset

In [57]:
summary_dataset.head()

,Unnamed: 0.1,Ind,title,text,summary,mbart_predictions,mt5_predictions,summarunner_predictions,llama_7b_predictions,starling_predictions,...,starling_predictions_rougeL,starling_predictions_rougeLsum,yagpt_predictions_rouge1,yagpt_predictions_rouge2,yagpt_predictions_rougeL,yagpt_predictions_rougeLsum,yagpt3_predictions_rouge1,yagpt3_predictions_rouge2,yagpt3_predictions_rougeL,yagpt3_predictions_rougeLsum
0,53,53,Названа опасность постоянно включенного Blueto...,Постоянно включенный Bluetooth на смартфоне гр...,Активированный в смартфоне Bluetooth может пре...,Постоянно включенный Bluetooth на смартфоне гр...,"Эксперты предупреждают о том, что отключать Bl...",постоянно включенный bluetooth на смартфоне гр...,"Bluetooth постоянно включен, это опасно, так к...",Постоянно включенный Bluetooth может создавать...,...,0.162162,0.162162,0.176101,0.025478,0.125786,0.138365,0.171429,0.011561,0.102857,0.114286
1,244,244,Колесникова проходит подозреваемой по делу о з...,Член президиума координационного совета оппози...,Члена президиума оппозиционного Координационно...,Член президиума координационного совета оппози...,Член президиума координационного совета оппози...,член президиума координационного совета оппози...,"Мария Колесникова, член президиума координацио...","Мария Колесникова, член президиума координацио...",...,0.256410,0.256410,0.310160,0.086486,0.235294,0.171123,0.257732,0.031250,0.164948,0.175258
2,260,260,Deutsche Bank: в мире наступает эпоха беспорядка,Аналитики Deutsche Bank обнародовали исследова...,В истории человечества наступает эпоха беспоря...,В 2020 году в мире наступит новая эпоха беспор...,Пандемия коронавируса привела к появлениям нов...,аналитики deutsche bank обнародовали исследова...,Аналитики Deutsche Bank предсказали наступлени...,"Аналитики Deutsche Bank предполагают, что 2020...",...,0.214286,0.214286,0.241546,0.058537,0.164251,0.164251,0.309677,0.078431,0.193548,0.245161
3,355,355,«Ъ»: Минтранс подготовил проект поправок к ПДД,ГИБДД совместно с Минтрансом разработала масшт...,Масштабный проект изменений в ПДД подготовили ...,ГИБДД совместно с Минтрансом разработала масшт...,В Госдуму вступили в силу поправки в правила д...,гибдд совместно с минтрансом разработала масшт...,ГИБДД и Минтранс разработали проект изменений ...,ГИБДД и Минтранс разработали проект изменений ...,...,0.174497,0.174497,0.239130,0.054945,0.173913,0.173913,0.317757,0.095238,0.224299,0.205607
4,653,653,Tesla выпустит бюджетный беспилотник в 2023 году,Вечером 22 сентября на конференции Battery Day...,Миллиардер Илон Маск пообещал представить бюдж...,На конференции Battery Day глава Tesla Илон Ма...,Илон Маск рассчитывает вывести на рынок бюджет...,вечером 22 сентября на конференции battery day...,Илон Маск представил планы Tesla по выходу на ...,"В ходе конференции Tesla Battery Day, Илон Мас...",...,0.198473,0.198473,0.321678,0.056738,0.223776,0.223776,0.285714,0.108434,0.190476,0.238095


Посчитаем статистики для представления о качестве датасета

In [58]:
from statistics import mean 
from collections import defaultdict
score_stats = defaultdict(dict) if (not "score_stats" in locals() and not "score_stats" in globals()) else score_stats
for filename in files:
    predictions_field = filename.split(".")[0]
    mean_rouge1 = mean(results[predictions_field][f"{predictions_field}_rouge1"])
    var_rouge1 = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_rouge1"])) - mean_rouge1 ** 2
    
    mean_rouge2 = mean(results[predictions_field][f"{predictions_field}_rouge2"])
    var_rouge2 = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_rouge2"])) - mean_rouge2 ** 2
    
    mean_rougeL = mean(results[predictions_field][f"{predictions_field}_rougeL"])
    var_rougeL = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_rougeL"])) - mean_rougeL ** 2
    
    mean_rougeLsum = mean(results[predictions_field][f"{predictions_field}_rougeLsum"])
    var_rougeLsum = mean(map(lambda x: x**2, results[predictions_field][f"{predictions_field}_rougeLsum"])) - mean_rougeLsum ** 2
    score_stats[predictions_field].update({"mean_rouge1": mean_rouge1, "var_rouge1": var_rouge1, "mean_rouge2": mean_rouge2, "var_rouge2": var_rouge2, "mean_rougeL": mean_rougeL, "var_rougeL": var_rougeL, "mean_rougeLsum": mean_rougeLsum, "var_rougeLsum": var_rougeLsum})
print(dict(score_stats))

{'mbart_predictions': {'mean_precision': 0.6900501682208134, 'var_precision': 0.0019415853485905687, 'mean_recall': 0.6960230689782363, 'var_recall': 0.0025041195684459283, 'mean_f1': 0.6922733288544874, 'var_f1': 0.00170587457440724, 'mean_rouge1': 0.3132584410306658, 'var_rouge1': 0.008066642067746585, 'mean_rouge2': 0.10460121341018001, 'var_rouge2': 0.007166239563576184, 'mean_rougeL': 0.22920292756039287, 'var_rougeL': 0.007787845278402608, 'mean_rougeLsum': 0.22920292756039287, 'var_rougeLsum': 0.007787845278402608}, 'mt5_predictions': {'mean_precision': 0.7098548559042124, 'var_precision': 0.003583850681936007, 'mean_recall': 0.6099580485087175, 'var_recall': 0.0028486138733002653, 'mean_f1': 0.6551174383897048, 'var_f1': 0.002486519453542002, 'mean_rouge1': 0.18770184990638486, 'var_rouge1': 0.008669013764652747, 'mean_rouge2': 0.055957208707190335, 'var_rouge2': 0.004357179302824177, 'mean_rougeL': 0.15503712316282722, 'var_rougeL': 0.0064435302208476555, 'mean_rougeLsum': 0.1

In [59]:
stats_pd = pd.DataFrame(score_stats)

In [60]:
stats_pd.head(len(stats_pd))

,mbart_predictions,mt5_predictions,summarunner_predictions,llama_7b_predictions,starling_predictions,yagpt_predictions,yagpt3_predictions
mean_precision,0.690050,0.709855,0.663267,0.644400,0.665735,0.657500,0.659426
var_precision,0.001942,0.003584,0.002033,0.000821,0.002084,0.000793,0.001332
mean_recall,0.696023,0.609958,0.676482,0.722376,0.706216,0.714787,0.731001
var_recall,0.002504,0.002849,0.002610,0.001855,0.001771,0.001589,0.002079
mean_f1,0.692273,0.655117,0.669170,0.680531,0.684643,0.684335,0.692904
var_f1,0.001706,0.002487,0.001854,0.000826,0.001501,0.000723,0.001329
mean_rouge1,0.313258,0.187702,0.277396,0.236332,0.279708,0.237622,0.271905
var_rouge1,0.008067,0.008669,0.009279,0.003580,0.005227,0.003213,0.004402
mean_rouge2,0.104601,0.055957,0.073982,0.067198,0.078820,0.069057,0.087630
var_rouge2,0.007166,0.004357,0.007102,0.001667,0.003717,0.001433,0.004131


In [61]:
summary_dataset.to_csv("summary_dataset_with_bertscore_bleu_rouge.csv", index=False)

## METEOR

https://huggingface.co/spaces/evaluate-metric/meteor

In [62]:
import pandas as pd
summary_dataset = pd.read_csv("summary_dataset_with_bertscore_bleu_rouge.csv")
files = [
    "mbart_predictions.txt",
    "mt5_predictions.txt",
    "summarunner_predictions.txt",
    "llama_7b_predictions.csv",
    "starling_predictions.csv",
    "yagpt_predictions.csv",
    "yagpt3_predictions.csv"
]

In [63]:
summary_dataset.columns

Index(['Unnamed: 0.1', 'Ind', 'title', 'text', 'summary', 'mbart_predictions',
       'mt5_predictions', 'summarunner_predictions', 'llama_7b_predictions',
       'starling_predictions', 'yagpt_predictions', 'yagpt3_predictions',
       'mbart_predictions_bertscore_precision',
       'mbart_predictions_bertscore_recall', 'mbart_predictions_bertscore_f1',
       'mt5_predictions_bertscore_precision',
       'mt5_predictions_bertscore_recall', 'mt5_predictions_bertscore_f1',
       'summarunner_predictions_bertscore_precision',
       'summarunner_predictions_bertscore_recall',
       'summarunner_predictions_bertscore_f1',
       'llama_7b_predictions_bertscore_precision',
       'llama_7b_predictions_bertscore_recall',
       'llama_7b_predictions_bertscore_f1',
       'starling_predictions_bertscore_precision',
       'starling_predictions_bertscore_recall',
       'starling_predictions_bertscore_f1',
       'yagpt_predictions_bertscore_precision',
       'yagpt_predictions_bertscore_

In [64]:
from tqdm import tqdm
from threading import Thread
from evaluate import load
import os, contextlib

import nltk
nltk.download('wordnet')


def calculate_meteor(result_list, predictions, references, thread_no, step):
    meteor = None
    # with open(os.devnull, 'w') as devnull:
    #     with contextlib.redirect_stdout(devnull), contextlib.redirect_stderr(devnull):
    
    meteor = load('meteor')
    index = thread_no
    disable_tqdm = (thread_no != 0)
    
    
    with tqdm(total=len(predictions), disable=disable_tqdm) as pbar:
        pbar.update(index)
        while index < len(predictions):
            result_list[index] = meteor.compute(predictions=[predictions[index]], references=[references[index]])
            index += step
            pbar.update(step)
    print(f"Thread {thread_no} finished")

results = {}
references = list(summary_dataset["summary"])
for filename in files:
    predictions_field = filename.split(".")[0]
    predictions = list(summary_dataset[predictions_field])
    results[predictions_field] = [0] * len(predictions)
    
    thread_num = 32
    thread_pool = []
    for thread in range(thread_num):
        thread_pool.append(Thread(target=calculate_meteor, args=[results[predictions_field], predictions, references, thread, thread_num], daemon=True))
        thread_pool[-1].start()

    for thread in range(thread_num):
        thread_pool[thread].join()

[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up

Thread 11 finished
Thread 6 finished
Thread 3 finished


  0%|                                                                   | 0/65 [00:00<?, ?it/s][nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data]   Package omw-1.4 is already up-to-date!
96it [00:00, 4580.24it/s]                                                                      


Thread 0 finished
Thread 12 finished
Thread 15 finished
Thread 20 finished
Thread 28 finished
Thread 13 finished
Thread 31 finished
Thread 17 finished
Thread 18 finished
Thread 1 finished
Thread 16 finished
Thread 27 finished
Thread 4 finished
Thread 19 finished


[nltk_data]   Package omw-1.4 is already up-to-date!


...

Thread 3 finished


  0%|                                                                   | 0/65 [00:00<?, ?it/s][nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/skatori/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/skatori/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skato

Thread 31 finished
Thread 20 finished
Thread 28 finished
Thread 16 finished
Thread 1 finished
Thread 0 finished
Thread 23 finished
Thread 8 finished
Thread 19 finished
Thread 13 finished
Thread 26 finished
Thread 24 finished
Thread 25 finished
Thread 18 finished


[nltk_data] Downloading package wordnet to /home/skatori/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/skatori/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/skatori/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Thread 5 finished
Thread 7 finished
Thread 30 finished
Thread 10 finished


In [65]:
print(results)

{'mbart_predictions': [{'meteor': 0.1264591439688716}, {'meteor': 0.24882943143812708}, {'meteor': 0.30963082823924243}, {'meteor': 0.3063148788927336}, {'meteor': 0.09889056691449813}, {'meteor': 0.21086415830573935}, {'meteor': 0.16566965589606567}, {'meteor': 0.17318553669684708}, {'meteor': 0.4043802922945224}, {'meteor': 0.2147186147186147}, {'meteor': 0.4260436681222708}, {'meteor': 0.27991741443659174}, {'meteor': 0.27381806227960076}, {'meteor': 0.254421768707483}, {'meteor': 0.5005431727775856}, {'meteor': 0.33779877260981916}, {'meteor': 0.13901212658976636}, {'meteor': 0.18330585509312894}, {'meteor': 0.26894287224865737}, {'meteor': 0.2222222222222222}, {'meteor': 0.14947216465628707}, {'meteor': 0.23696177609297744}, {'meteor': 0.28956422018348627}, {'meteor': 0.08445945945945944}, {'meteor': 0.21568906605922553}, {'meteor': 0.13054830287206265}, {'meteor': 0.26161634244341014}, {'meteor': 0.21879634206623827}, {'meteor': 0.1358234295415959}, {'meteor': 0.1655676959619953}

In [66]:
for key in results.keys():
    results[key] = {f"{key}_meteor": [val["meteor"] if not isinstance(val, int) else 0 for val in results[key]]}
    print(key, len(results[key]))

mbart_predictions 1
mt5_predictions 1
summarunner_predictions 1
llama_7b_predictions 1
starling_predictions 1
yagpt_predictions 1
yagpt3_predictions 1


In [67]:
new_data = {}
for data_key in results.keys():
    for metric_key in results[data_key].keys():
        if "hashcode" in metric_key:
            continue
        new_data[metric_key] = pd.Series(results[data_key][metric_key])
new_data_df = pd.DataFrame(new_data)
new_data_df.head()

,mbart_predictions_meteor,mt5_predictions_meteor,summarunner_predictions_meteor,llama_7b_predictions_meteor,starling_predictions_meteor,yagpt_predictions_meteor,yagpt3_predictions_meteor
0,0.126459,0.053648,0.168245,0.207257,0.127202,0.168304,0.139373
1,0.248829,0.101734,0.263807,0.231588,0.310913,0.327753,0.224195
2,0.309631,0.086664,0.314012,0.262029,0.238650,0.295642,0.311487
3,0.306315,0.082645,0.233184,0.256739,0.270406,0.338463,0.296722
4,0.098891,0.100573,0.146562,0.227205,0.193430,0.202198,0.251442


In [68]:
new_summary_dataset = pd.concat([summary_dataset, new_data_df.set_index(summary_dataset.index)], axis=1)
new_summary_dataset.head()

,Unnamed: 0.1,Ind,title,text,summary,mbart_predictions,mt5_predictions,summarunner_predictions,llama_7b_predictions,starling_predictions,...,yagpt3_predictions_rouge2,yagpt3_predictions_rougeL,yagpt3_predictions_rougeLsum,mbart_predictions_meteor,mt5_predictions_meteor,summarunner_predictions_meteor,llama_7b_predictions_meteor,starling_predictions_meteor,yagpt_predictions_meteor,yagpt3_predictions_meteor
0,53,53,Названа опасность постоянно включенного Blueto...,Постоянно включенный Bluetooth на смартфоне гр...,Активированный в смартфоне Bluetooth может пре...,Постоянно включенный Bluetooth на смартфоне гр...,"Эксперты предупреждают о том, что отключать Bl...",постоянно включенный bluetooth на смартфоне гр...,"Bluetooth постоянно включен, это опасно, так к...",Постоянно включенный Bluetooth может создавать...,...,0.011561,0.102857,0.114286,0.126459,0.053648,0.168245,0.207257,0.127202,0.168304,0.139373
1,244,244,Колесникова проходит подозреваемой по делу о з...,Член президиума координационного совета оппози...,Члена президиума оппозиционного Координационно...,Член президиума координационного совета оппози...,Член президиума координационного совета оппози...,член президиума координационного совета оппози...,"Мария Колесникова, член президиума координацио...","Мария Колесникова, член президиума координацио...",...,0.031250,0.164948,0.175258,0.248829,0.101734,0.263807,0.231588,0.310913,0.327753,0.224195
2,260,260,Deutsche Bank: в мире наступает эпоха беспорядка,Аналитики Deutsche Bank обнародовали исследова...,В истории человечества наступает эпоха беспоря...,В 2020 году в мире наступит новая эпоха беспор...,Пандемия коронавируса привела к появлениям нов...,аналитики deutsche bank обнародовали исследова...,Аналитики Deutsche Bank предсказали наступлени...,"Аналитики Deutsche Bank предполагают, что 2020...",...,0.078431,0.193548,0.245161,0.309631,0.086664,0.314012,0.262029,0.238650,0.295642,0.311487
3,355,355,«Ъ»: Минтранс подготовил проект поправок к ПДД,ГИБДД совместно с Минтрансом разработала масшт...,Масштабный проект изменений в ПДД подготовили ...,ГИБДД совместно с Минтрансом разработала масшт...,В Госдуму вступили в силу поправки в правила д...,гибдд совместно с минтрансом разработала масшт...,ГИБДД и Минтранс разработали проект изменений ...,ГИБДД и Минтранс разработали проект изменений ...,...,0.095238,0.224299,0.205607,0.306315,0.082645,0.233184,0.256739,0.270406,0.338463,0.296722
4,653,653,Tesla выпустит бюджетный беспилотник в 2023 году,Вечером 22 сентября на конференции Battery Day...,Миллиардер Илон Маск пообещал представить бюдж...,На конференции Battery Day глава Tesla Илон Ма...,Илон Маск рассчитывает вывести на рынок бюджет...,вечером 22 сентября на конференции battery day...,Илон Маск представил планы Tesla по выходу на ...,"В ходе конференции Tesla Battery Day, Илон Мас...",...,0.108434,0.190476,0.238095,0.098891,0.100573,0.146562,0.227205,0.193430,0.202198,0.251442


In [69]:
summary_dataset = pd.DataFrame(new_summary_dataset)
del new_summary_dataset

In [70]:
summary_dataset.head()

,Unnamed: 0.1,Ind,title,text,summary,mbart_predictions,mt5_predictions,summarunner_predictions,llama_7b_predictions,starling_predictions,...,yagpt3_predictions_rouge2,yagpt3_predictions_rougeL,yagpt3_predictions_rougeLsum,mbart_predictions_meteor,mt5_predictions_meteor,summarunner_predictions_meteor,llama_7b_predictions_meteor,starling_predictions_meteor,yagpt_predictions_meteor,yagpt3_predictions_meteor
0,53,53,Названа опасность постоянно включенного Blueto...,Постоянно включенный Bluetooth на смартфоне гр...,Активированный в смартфоне Bluetooth может пре...,Постоянно включенный Bluetooth на смартфоне гр...,"Эксперты предупреждают о том, что отключать Bl...",постоянно включенный bluetooth на смартфоне гр...,"Bluetooth постоянно включен, это опасно, так к...",Постоянно включенный Bluetooth может создавать...,...,0.011561,0.102857,0.114286,0.126459,0.053648,0.168245,0.207257,0.127202,0.168304,0.139373
1,244,244,Колесникова проходит подозреваемой по делу о з...,Член президиума координационного совета оппози...,Члена президиума оппозиционного Координационно...,Член президиума координационного совета оппози...,Член президиума координационного совета оппози...,член президиума координационного совета оппози...,"Мария Колесникова, член президиума координацио...","Мария Колесникова, член президиума координацио...",...,0.031250,0.164948,0.175258,0.248829,0.101734,0.263807,0.231588,0.310913,0.327753,0.224195
2,260,260,Deutsche Bank: в мире наступает эпоха беспорядка,Аналитики Deutsche Bank обнародовали исследова...,В истории человечества наступает эпоха беспоря...,В 2020 году в мире наступит новая эпоха беспор...,Пандемия коронавируса привела к появлениям нов...,аналитики deutsche bank обнародовали исследова...,Аналитики Deutsche Bank предсказали наступлени...,"Аналитики Deutsche Bank предполагают, что 2020...",...,0.078431,0.193548,0.245161,0.309631,0.086664,0.314012,0.262029,0.238650,0.295642,0.311487
3,355,355,«Ъ»: Минтранс подготовил проект поправок к ПДД,ГИБДД совместно с Минтрансом разработала масшт...,Масштабный проект изменений в ПДД подготовили ...,ГИБДД совместно с Минтрансом разработала масшт...,В Госдуму вступили в силу поправки в правила д...,гибдд совместно с минтрансом разработала масшт...,ГИБДД и Минтранс разработали проект изменений ...,ГИБДД и Минтранс разработали проект изменений ...,...,0.095238,0.224299,0.205607,0.306315,0.082645,0.233184,0.256739,0.270406,0.338463,0.296722
4,653,653,Tesla выпустит бюджетный беспилотник в 2023 году,Вечером 22 сентября на конференции Battery Day...,Миллиардер Илон Маск пообещал представить бюдж...,На конференции Battery Day глава Tesla Илон Ма...,Илон Маск рассчитывает вывести на рынок бюджет...,вечером 22 сентября на конференции battery day...,Илон Маск представил планы Tesla по выходу на ...,"В ходе конференции Tesla Battery Day, Илон Мас...",...,0.108434,0.190476,0.238095,0.098891,0.100573,0.146562,0.227205,0.193430,0.202198,0.251442


Посчитаем статистики для представления о качестве датасета

In [71]:
from collections import defaultdict

score_stats = defaultdict(dict) if (not "score_stats" in locals() and not "score_stats" in globals()) else score_stats
for filename in files:
    predictions_field = filename.split(".")[0]
    score_stats[predictions_field].update({f"meteor": sum(results[predictions_field][f"{predictions_field}_meteor"]) / len(results[predictions_field][f"{predictions_field}_meteor"])})

In [72]:
stats_pd = pd.DataFrame(score_stats)

In [73]:
stats_pd.head(len(stats_pd))

,mbart_predictions,mt5_predictions,summarunner_predictions,llama_7b_predictions,starling_predictions,yagpt_predictions,yagpt3_predictions
mean_precision,0.690050,0.709855,0.663267,0.644400,0.665735,0.657500,0.659426
var_precision,0.001942,0.003584,0.002033,0.000821,0.002084,0.000793,0.001332
mean_recall,0.696023,0.609958,0.676482,0.722376,0.706216,0.714787,0.731001
var_recall,0.002504,0.002849,0.002610,0.001855,0.001771,0.001589,0.002079
mean_f1,0.692273,0.655117,0.669170,0.680531,0.684643,0.684335,0.692904
var_f1,0.001706,0.002487,0.001854,0.000826,0.001501,0.000723,0.001329
mean_rouge1,0.313258,0.187702,0.277396,0.236332,0.279708,0.237622,0.271905
var_rouge1,0.008067,0.008669,0.009279,0.003580,0.005227,0.003213,0.004402
mean_rouge2,0.104601,0.055957,0.073982,0.067198,0.078820,0.069057,0.087630
var_rouge2,0.007166,0.004357,0.007102,0.001667,0.003717,0.001433,0.004131


In [74]:
summary_dataset.to_csv("summary_dataset_with_bertscore_bleu_rouge_meteor.csv", index=False)